In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-19 10:41:50.247537: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-19 10:41:50.305165: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [2]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [3]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
target_test_loader = DataLoader(test_target_data, batch_size=32)

In [4]:
from adapters import AutoAdapterModel
from tqdm import tqdm

from transformers import AutoConfig

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
    num_labels=3,
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
    config=mdlcfg,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [5]:
domain_adapter_coral = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/domain-adapter-coral")


In [6]:
from transformers import AdamW
adapter_name= "stacked-adapter-task"
model.add_adapter(adapter_name, config="seq_bn")
model.add_classification_head(
    adapter_name,
    num_labels=3,
  )
model.train_adapter([adapter_name])
fn.print_trainable_parameters(model)

trainable params: 1662813 || all params: 68472957 || trainable%: 2.428422946594814


In [7]:
from adapters.composition import Stack
model.active_adapters = Stack(domain_adapter_coral, adapter_name)

In [8]:
model.adapter_summary(as_dict=True)

[{'name': 'domain-adapter-coral',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 447264,
  'train': False,
  '%param': 0.6739671334336303},
 {'name': 'stacked-adapter-task',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 447264,
  'train': True,
  '%param': 0.6739671334336303},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [9]:
fn.print_trainable_parameters(model)
model.active_adapters

trainable params: 1662813 || all params: 68472957 || trainable%: 2.428422946594814


Stack[domain-adapter-coral, stacked-adapter-task]

In [10]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.31983805668016196
F1 score before adaptation: 0.19905971675655318


In [11]:
reload(config)
reload(fn)
trainer = fn.train_model(model,"stacked",source_data,source_data_eval)

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/9380 [00:00<?, ?it/s]

{'loss': 1.0576, 'learning_rate': 0.0001, 'epoch': 0.21}


  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.9236879348754883, 'eval_accuracy': 0.534919028340081, 'eval_f1': 0.5302478957015749, 'eval_precision': 0.5823660893334186, 'eval_recall': 0.534919028340081, 'eval_runtime': 2.6878, 'eval_samples_per_second': 735.174, 'eval_steps_per_second': 11.534, 'epoch': 0.21}
{'loss': 0.8614, 'learning_rate': 9.436936936936938e-05, 'epoch': 0.43}


...

{'eval_loss': 0.7148787379264832, 'eval_accuracy': 0.7004048582995951, 'eval_f1': 0.7010183349459501, 'eval_precision': 0.7168658635301949, 'eval_recall': 0.7004048582995951, 'eval_runtime': 2.7375, 'eval_samples_per_second': 721.822, 'eval_steps_per_second': 11.324, 'epoch': 3.62}
{'loss': 0.6402, 'learning_rate': 4.279279279279279e-06, 'epoch': 3.84}


  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.7259374856948853, 'eval_accuracy': 0.6902834008097166, 'eval_f1': 0.6912942336496789, 'eval_precision': 0.7153403334931405, 'eval_recall': 0.6902834008097166, 'eval_runtime': 2.743, 'eval_samples_per_second': 720.379, 'eval_steps_per_second': 11.301, 'epoch': 3.84}


Overwriting existing adapter 'domain-adapter-coral'.
Overwriting existing adapter 'stacked-adapter-task'.
Overwriting existing head 'stacked-adapter-task'


{'train_runtime': 878.4441, 'train_samples_per_second': 341.572, 'train_steps_per_second': 10.678, 'train_loss': 0.7216633534380622, 'epoch': 4.0}


In [12]:
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7039473684210527
F1 score before adaptation: 0.7045184361608424


In [32]:
model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)

In [33]:
model.add_classification_head(
    "task-test-after-coral",
    num_labels=3,
  )

In [37]:
model.adapter_summary(as_dict=True)

[{'name': 'domain-adapter-coral',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 447264,
  'train': True,
  '%param': 0.6739671334336303},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [36]:
accuracy_before, f1_before = fn.evaluate_model(model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.34210526315789475
F1 score before adaptation: 0.2640755557195107
